In [9]:
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np 
from gensim.similarities import WmdSimilarity

In [2]:
## loading the pre trained model 

model = KeyedVectors.load_word2vec_format('nepali_embeddings_word2vec.txt',binary=False)

In [3]:
## loading the question,intent dataset 

doc = pd.read_csv('dataset_modified.txt')

doc.head()

,Sentence,Final Intent
0,तिम्रो नाम के हो,परि
1,तिमिलाई के भनेर चिन्छन अरुले,परि
2,तिमिलाई बोलाउनि नाम के हो,परि
3,म तिमिलाई के भनेर बोलाउ,परि
4,तिमिलाई के भनेर चिन्छन,परि


In [5]:
## loading the intent,response dataset

response = pd.read_csv('follow_up_qsn_v1.csv')
response.head()

,Intent,Count,Response
0,हुक्का,18,यहाँ हुक्का पाइन्छ। पान मिन्ट डबल एप्पल र स्ट्...
1,चाउमिन,17,यहाँ चाउमिन पाइन्छ र हामी सँग चिकेन चाउमिन बफ ...
2,पिजा,17,यहाँ पिजा पाइन्छ र हामी सँग चिज पिजा मसरूम पिज...
3,मोमो,17,यहाँ मोमो पाइन्छ र हामी सँग चिकेन मोमो बफ मोमो...
4,बफ चिल्ली,16,यहाँ बफ चिल्ली पाइन्छ। बफ चिल्ली लाई प्लेटको द...


In [77]:
val =response[response['Intent']=='पिजा'].Response.values
for x in val:
    print(x)
#response['Response'][val]

यहाँ पिजा पाइन्छ र हामी सँग चिज पिजा मसरूम पिजा चिकेन पिजा र मिक्स पिजा छ | पिजा बन्न बिस मिनेट लाग्छ|


In [6]:
doc['Final Intent'].value_counts()

हुक्का             18
चाउमिन             17
मोमो               17
पिजा               17
परि                16
चिकेन लेगपीस       16
प्याज पकौडा        16
चिकेन चिल्ली       16
बफ सुकुटी          16
चिकेन ललिपप        16
आलु पकौडा          16
बर्गर              16
बफ ससेज            16
चिकेन ससेज         16
चिकेन सुकुटी       16
बफ चिल्ली          16
चिसो               15
ससेज               12
सुकुटी             12
वाईफाई             11
मिक्स पिजा         10
अफर                10
चिकेन चाउमिन       10
चिकेन मोमो         10
मसरूम पिजा         10
बफ चाउमिन          10
भेज मोमो           10
पकौडा              10
भेज चाउमिन         10
चिकेन पिजा         10
बफ मोमो            10
चिज पिजा           10
सी मोमो             9
कोक                 9
वाईन                8
स्पेशल              8
रुसलान              8
खुकुरी              8
टुबोर्ग गोल्ड       8
टुबोर्ग स्ट्रोङ     8
फेन्टा              8
डिउ                 8
बियर                8
टुबोर्ग डिलक्स      8
स्प्राइट            8
अर्ना     

In [107]:
## create corpus of the sentences from document 

corpus = [sent for sent in doc['Sentence']]

In [129]:
## Creating a function to return the intent of the question

def intent_of_question(model,doc,sentence,response):
    instance = WmdSimilarity(corpus, model, num_best=5)
    similar = instance[sentence]
    list_of_intent=[]
    for index,score in similar:
        if score <0.75:
            list_of_intent.append(('अन्य',score))
        else:
            intent = doc['Final Intent'][index]
            list_of_intent.append((intent,score))
    return list_of_intent

In [132]:
## Creating a list of sentence for testing

list_of_sentence=['खुकुरी रम पाउला कि नाई','कति समय लाग्छ प्याज पकौडा बन्न','छुट छ','आज मौसम कस्तो छ','परि आज स्पेशल के हो','मसरूम पिजा छ','आज डिसकाउन्ट छ','वाईफाईको इस्पिड कस्तो छ']

## Calculating the response
for x in list_of_sentence:
    intnt=intent_of_question(model=model,doc=doc,sentence=x,response=response)
    
    maxm = 0
    context=''
    for intention,prob in intnt:
        if prob>maxm:
            maxm=prob
            context=intention
    result=response[response['Intent']==context].Response.values
    print(f'Question -->{x} \nAnswer -->{result}')

Question -->खुकुरी रम पाउला कि नाई 
Answer -->['यहाँ खुकुरी रम पाइन्छ। हामी सँग खुकुरी रम क्वाटर बोत्तल हाफ बोत्तल र फुल बोत्तल छ। क्वाटरको तिन सय पचास रुपैया पर्छ हाफको छ सय पचास रुपैया पर्छ र फुल खुकुरी रमको बाह्र सय रुपैया पर्छ।']
Question -->कति समय लाग्छ प्याज पकौडा बन्न 
Answer -->['यहाँ प्याज पकौडा पाइन्छ। प्याज पकौडालाई प्लेटको एक सय बिस रुपैया हो र बन्न पन्ध्र मिनेट समय लाग्छ।']
Question -->छुट छ 
Answer -->['अफर भनेर त केहि छैन तर थोरै डिस्काउन्ट हुन्छ |']
Question -->आज मौसम कस्तो छ 
Answer -->['सरि तपाइको प्रश्न यो रेस्टुरेन्ट सम्बन्धि हुनु पर्छ।']
Question -->परि आज स्पेशल के हो 
Answer -->['यो रेस्टुरेन्टमा स्पेशल पाउनि भनेको चिकेन सुकुटी चिकेन ललिपप र बफ चिल्ली हो | ']
Question -->मसरूम पिजा छ 
Answer -->['यहाँ मसरुम पिजा पाइन्छ| स्माल साइजको तिन सय मिडियम साइजको चार सय र लार्ज साइजको पाँच सय|']
Question -->आज डिसकाउन्ट छ 
Answer -->['अफर भनेर त केहि छैन तर थोरै डिस्काउन्ट हुन्छ |']
Question -->वाईफाईको इस्पिड कस्तो छ 
Answer -->['वाईफाईको सुविधा छ र वाईफाईको स्पीड पनि र